In [43]:
from tensorflow.keras import Sequential
from  tensorflow.keras.layers import Dense, LSTM, GRU

import pandas as pd
import numpy as np 



In [44]:
window_size = 7

df = pd.read_csv("smart_power.csv")

X , y = [] , []
for i in range(len(df) - window_size):
    X.append(df["Electricity_Demand_MW"][i : i + window_size])
    y.append(df["Electricity_Demand_MW"][i + window_size])

X = np.array(X)
y = np.array(y)

In [45]:
model_1 = Sequential([
    LSTM(32, activation="tanh", 
    recurrent_activation="sigmoid", 
    input_shape=(window_size, 1)),
    Dense(1, activation="linear")])

model_1.summary()

d:\Python Programs\DevEnv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.1)  

In [47]:
model_1.compile(optimizer=optimizer, loss="mse", metrics=["mae"])


In [48]:
model_1.fit(X, y, epochs=1000, batch_size=32, validation_split=.25)

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step - loss: 340463.7500 - mae: 575.9535 - val_loss: 670297.8125 - val_mae: 817.7111
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 336944.5312 - mae: 572.9259 - val_loss: 666547.6250 - val_mae: 815.4146
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 334307.4062 - mae: 570.6187 - val_loss: 662749.8750 - val_mae: 813.0827
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 331660.6875 - mae: 568.2820 - val_loss: 658939.0625 - val_mae: 810.7358
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 329040.2812 - mae: 565.9315 - val_loss: 655126.3750 - val_mae: 808.3810
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 326338.9688 - mae: 563.5785 - val_loss: 651328.1875 - val_mae: 806.0283
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 323692.5312 - mae: 561.2253 - val_loss: 647538.6875 - val_mae: 803.6741
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 321055.7500 - mae: 558.

In [49]:
model_2 = Sequential([
    GRU(32 ,
        activation="tanh",
        recurrent_activation = "sigmoid",
        input_shape=(window_size, 1)
        ),
        Dense(1)
    ])

In [54]:
from sklearn.preprocessing import StandardScaler


split = int(len(X) * 0.8)
X_tr, X_val = X[:split], X[split:]
y_tr, y_val = y[:split], y[split:]


y_tr = y_tr.reshape(-1, 1)   # (N, 1)
y_val = y_val.reshape(-1, 1) 


x_scaler = StandardScaler().fit(X_tr)
y_scaler = StandardScaler().fit(y_tr)


X_tr = x_scaler.transform(X_tr)
X_val = x_scaler.transform(X_val)
y_tr = y_scaler.transform(y_tr)
y_val = y_scaler.transform(y_val)

X_tr = X_tr[..., None]
X_val = X_val[..., None]


In [55]:
model_1.fit(X_tr,y_tr, epochs=200, validation_data=(X_val,y_val))

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 6586.2241 - mae: 59.5582 - val_loss: 203.5182 - val_mae: 13.6455
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 748.2238 - mae: 21.5237 - val_loss: 38.6193 - val_mae: 6.1813
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 12.5268 - mae: 3.4339 - val_loss: 33.9458 - val_mae: 5.8094
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 9.6439 - mae: 2.9652 - val_loss: 14.9975 - val_mae: 3.8494
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2.4744 - mae: 1.3212 - val_loss: 3.7241 - val_mae: 1.8858
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 3.9101 - mae: 1.4634 - val_loss: 1.0898 - val_mae: 0.9714
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.4552 - mae: 1.0734 - val_loss: 0.9282 - val_mae: 0.8849
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.5864 - mae: 0.6357 - val_loss: 1.2586 - val_mae: 1.0524
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 